<div style="text-align:center">
<img src="img/nlopt2.gif" width="600">
</div>

# <b> Summary </b>
1. <b> French Gemini AI : Répartition de la charge électrique (Economic Dispatch) - Other language test</b>
    - Abstract
    - Instance
    - Math model
    - Cplex solution
2. <b> ChatGpt AI : Factory Electricity load balancing</b>
    - Abstract
    - Instance
    - Math model
    - Cplex solution


# <b> 1. Gemini : Répartition de la charge électrique (Economic Dispatch)</b>



<div style="text-align:center">
<img src="img/top-10-nuclear-power-companies.png" width="400">
</div>

<div style="text-align:center">
<img src="img/gemini.png" width="200"/>
</div>

## <b>Abstract</b>
L'optimisation quadratique (QP) est extrêmement courante dans le monde réel, notamment pour équilibrer les coûts et la performance. Un exemple parfait est la répartition de la charge électrique (Economic Dispatch).

Imaginons que vous dirigiez une petite centrale électrique avec deux générateurs. Votre but est de produire une quantité précise d'électricité au coût le plus bas possible.

Le Problème : Pourquoi "Quadratique" ?

Le coût de fonctionnement d'un générateur n'est pas linéaire. Plus on le sollicite, plus il perd en efficacité (chaleur, usure), ce qui fait grimper le coût de façon exponentielle. On modélise souvent cela par une fonction du second degré :

Cout=ax²+bx+c

## <b>Instance</b>


<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-30-55 Google Gemini.png" width="400">
</div>

## <b>Math model</b>


<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-40-22 Google Gemini.png">
</div>

<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-44-38 Google Gemini.png">
</div>

<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-45-39 Google Gemini.png">
</div>






## <b>Cplex solution</b>
<div style="text-align:center">
<img src="img/IBM_CPLEX_0.png">
</div>

In [33]:
from docplex.mp.model import Model

# 1. Créer le modèle
mdl = Model(name='Optimisation_Energie')

# 2. Variables de décision (Puissance produite par chaque générateur en MW)
# On définit des bornes (ex: min 10MW, max 200MW)
x1 = mdl.continuous_var(lb=10, ub=200, name='gen1')
x2 = mdl.continuous_var(lb=10, ub=200, name='gen2')

# 3. Fonction Objectif : Minimiser le coût total (Quadratique)
# Coût = (0.5*x1^2 + 2*x1) + (0.25*x2^2 + 8*x2)
mdl.minimize(0.5*x1**2 + 2*x1 + 0.25*x2**2 + 8*x2)

# 4. Contrainte : Satisfaire la demande de 150 MW
mdl.add_constraint(x1 + x2 == 150, "Demande_Totale")

# 5. Résolution
solution = mdl.solve()

# 6. Affichage des résultats
if solution:
    print(f"Statut : {mdl.get_solve_status()}")
    print(f"Puissance Générateur 1 : {solution[x1]:.2f} MW")
    print(f"Puissance Générateur 2 : {solution[x2]:.2f} MW")
    print(f"Coût Total : {solution.objective_value:.2f} €")
else:
    print("Aucune solution trouvée.")

Statut : JobSolveStatus.OPTIMAL_SOLUTION
Puissance Générateur 1 : 54.00 MW
Puissance Générateur 2 : 96.00 MW
Coût Total : 4638.00 €


Pourquoi c'est intelligent ?

Si le coût était linéaire, CPLEX choisirait simplement le générateur le moins cher et le ferait tourner au maximum. Ici, comme le coût est quadratique, CPLEX calcule le point d'équilibre où l'augmentation du coût marginal du premier générateur devient plus élevée que celle du deuxième.

Dans cet exemple, CPLEX va répartir la charge de manière à ce que les "pentes" des coûts soient égales, minimisant ainsi la facture totale.

Ce que CPLEX apporte de plus

Dans un vrai scénario industriel, CPLEX gérerait en plus :

    Les pertes en ligne (la résistance des câbles).

    Les émissions de CO2 (une autre fonction quadratique à minimiser).

    Le démarrage/arrêt (variables binaires, ce qui transforme le problème en MIQP - Mixed Integer Quadratic Programming).

# <b> 2. ChatGpt AI : Factory Electricity load balancing</b>

<div style="text-align:center">
<img src="img/GalesbergSubstation.jpg" width="400" height="400">
</div>

<div style="text-align:center">
<img src="img/chatgpt.png" width="200"/>
</div>


## Abstract

<i>" Can you do a real life simple non linear example of Electric load balancing in a factory, using cplex ?"</i>

Yep — let’s do a real-life, simple, non-linear electric load balancing example that actually makes sense on a factory floor, and then show how you’d model it in CPLEX (MIQP / QP).

I’ll keep it concrete and practical, not textbook-fluffy.

## Instance : Real-life factory story (simple + realistic)
Imagine a factory with 3 machines that can be run during a given hour:

| Machine | Description         | Power (kW) | Notes                    |
| ------- | ------------------- | ---------- | ------------------------ |
| M1      | Cutting machine     | up to 50   | High efficiency          |
| M2      | Welding machine     | up to 40   | Medium efficiency        |
| M3      | Heat treatment oven | up to 60   | Inefficient at high load |

Reality constraints

- The factory must process 100 kW total load in this hour.

- Each machine has a maximum capacity.

- Electric losses and wear increase non-linearly when a machine is heavily loaded.

- Electricity tariff penalizes peaks → quadratic cost.

👉 This is why it’s non-linear.

## Math model

2️⃣ Decision variables

Let:

$$
x1​,x2​,x3 ​= power\thinspace  assigned\thinspace  to\thinspace  machines\thinspace  M1, M2, M3 (kW)
$$

3️⃣ Non-linear cost (very realistic)

Each machine has a quadratic operating cost:
$$
Cost_{i}​=a_{i}​x_{i}^2​+b_{i}x_{i}​
$$

Why quadratic?

- Higher current → higher losses

- Thermal stress

- Maintenance degradation

Exemple coefficients:

| Machine | (a_i) (loss factor) | (b_i) (energy price) |
| ------- | ------------------- | -------------------- |
| M1      | 0.01                | 5                    |
| M2      | 0.015               | 4                    |
| M3      | 0.03                | 3                    |

👉 M3 looks cheap per kW, but becomes very expensive when heavily loaded.

4️⃣ Optimization model (math)

<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 22-34-22 Electric Load Balancing CPLEX.png">
</div>




## Cplex solution

<div style="text-align:center">
<img src="img/IBM_CPLEX_0.png">
</div>

In [34]:
from docplex.mp.model import Model

# Create model
mdl = Model(name='Factory_Load_Balancing')

# Decision variables
x1 = mdl.continuous_var(lb=0, ub=50, name='M1')
x2 = mdl.continuous_var(lb=0, ub=40, name='M2')
x3 = mdl.continuous_var(lb=0, ub=60, name='M3')

# Objective function (quadratic)
mdl.minimize(
    0.01 * x1**2 + 5 * x1 +
    0.015 * x2**2 + 4 * x2 +
    0.03 * x3**2 + 3 * x3
)

# Demand constraint
mdl.add_constraint(x1 + x2 + x3 == 100)

# Solve
solution = mdl.solve()

# Print solution
if solution:
    print("Optimal Load Allocation:")
    print(f"M1: {x1.solution_value:.2f} kW")
    print(f"M2: {x2.solution_value:.2f} kW")
    print(f"M3: {x3.solution_value:.2f} kW")
    print(f"Total Cost: {solution.objective_value:.2f}")
else:
    print("No solution found")

Optimal Load Allocation:
M1: 20.00 kW
M2: 40.00 kW
M3: 40.00 kW
Total Cost: 456.00


6️⃣ What kind of solution you’ll see (intuition)

CPLEX will:

- Load M1 and M2 more evenly

- Avoid pushing M3 too hard despite cheaper base price

- Smooth the load → no peaks

7️⃣ Why this is a non-linear example (important)

| Feature                 | Linear   | This model          |
| ----------------------- | -------- | ------------------- |
| Cost per kW             | Constant | Increases with load |
| Peak penalties          | ❌        | ✅                   |
| Loss modeling           | ❌        | ✅                   |
| Realistic for factories | ❌        | ✅                   |

8️⃣ Want to level this up?

I can easily extend this to:

🔌 Time-of-use tariffs

⚡ Transformer capacity limits

🏭 On/Off machines (binary + MIQP)

🔋 Battery + renewable integration

📉 Demand response incentives

Just tell me how real you want it 😄